Import Library

In [10]:
import pandas as pd
import chardet

In [11]:
with open('stat_problem.csv', 'rb') as file:
    result = chardet.detect(file.read())
    print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [12]:
data = pd.read_csv('stat_problem.csv', sep=',', encoding='cp1252')
print(data.head()) #Head hanya menampilkan 5 Index awal

                                             problem        answer
0  Alice has 2 kids and one of them is a girl. Wh...  3.330000e-01
1  A fair six-sided die is rolled twice. What is ...  1.388889e-01
2  Consider a tetrahedral , A tetrahedral die has...  3.750000e-01
3  Anita randomly picks 4 cards from a deck of 52...  5.271878e+10
4  If you dealt 13 cards, what is the probability...  7.692308e-02


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch